# Task 3.7: Bayesian Optimization for XGBoost

## Objective

The goal of this task is to use **Bayesian Optimization** (via Optuna) to find optimal XGBoost hyperparameters. We will compare the results with the GridSearchCV approach used in Week 2 to demonstrate the advantages of Bayesian optimization.

## Why Bayesian Optimization?

In Week 2, we used **GridSearchCV** which exhaustively searches through a predefined grid of hyperparameters. While effective, this approach has limitations:

| Aspect | GridSearchCV | Bayesian Optimization |
|--------|--------------|----------------------|
| Search Strategy | Exhaustive (tries all combinations) | Intelligent (learns from previous trials) |
| Efficiency | O(n^k) where k = number of parameters | Typically finds optimum in fewer trials |
| Continuous Parameters | Must discretize | Handles continuous ranges naturally |
| Adaptability | Fixed grid | Adapts search based on results |
| Scalability | Poor for large search spaces | Excellent for large search spaces |

## How Bayesian Optimization Works

1. **Surrogate Model:** Builds a probabilistic model of the objective function (e.g., Gaussian Process or Tree-structured Parzen Estimator)
2. **Acquisition Function:** Decides which hyperparameters to try next based on:
   - **Exploitation:** Areas where the model predicts good performance
   - **Exploration:** Areas with high uncertainty
3. **Iterative Improvement:** Each trial updates the surrogate model, making future predictions more accurate

## Optuna Framework

We use **Optuna** because:
- State-of-the-art Bayesian optimization library
- Uses Tree-structured Parzen Estimator (TPE) by default
- Supports pruning (early stopping of unpromising trials)
- Easy integration with XGBoost
- Excellent visualization tools

## Week 2 XGBoost Results (Baseline)

From Task 2.3, our XGBoost model achieved:
- **Test Accuracy:** 99.40%
- **F1-Score (Macro):** 99.40%

Our goal is to see if Bayesian optimization can match or improve these results more efficiently.

## Step 1: Environment Setup and Data Loading

### Libraries Used:
- **optuna:** Bayesian optimization framework
- **xgboost:** XGBoost classifier
- **sklearn:** For metrics and cross-validation
- **matplotlib:** For visualization

### Installation Note:
If Optuna is not installed, run: `pip install optuna`

In [ ]:
# Install optuna if not available
# !pip install optuna

import pandas as pd
import numpy as np
import optuna
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pickle
import time
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("Libraries imported successfully!")
print(f"Optuna version: {optuna.__version__}")

In [ ]:
# Load data
X_train = pd.read_csv('../../data/processed/X_train_scaled.csv')
X_test = pd.read_csv('../../data/processed/X_test_scaled.csv')
y_train = pd.read_csv('../../data/processed/y_train.csv')
y_test = pd.read_csv('../../data/processed/y_test.csv')

# Remove ID columns if present
if 'id' in X_train.columns:
    X_train = X_train.drop('id', axis=1)
if 'id' in X_test.columns:
    X_test = X_test.drop('id', axis=1)

# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train['value_category'])
y_test_encoded = label_encoder.transform(y_test['value_category'])

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")
print(f"\nTarget classes: {label_encoder.classes_}")
print(f"\nTarget distribution (Training):")
unique, counts = np.unique(y_train_encoded, return_counts=True)
for val, count in zip(unique, counts):
    print(f"  Class {val} ({label_encoder.classes_[val]}): {count} ({count/len(y_train_encoded)*100:.1f}%)")

## Step 2: Load Week 2 XGBoost Results (Baseline)

Before running Bayesian optimization, let's load our Week 2 results to establish a baseline for comparison.

### Week 2 Hyperparameters (from Task 2.3):
- learning_rate = 0.1
- max_depth = 6
- n_estimators = 200
- subsample = 0.8
- colsample_bytree = 0.8

These were manually selected based on common best practices. Bayesian optimization will search a wider range to potentially find better values.

In [ ]:
# Load Week 2 XGBoost results
try:
    week2_results = pd.read_csv('../../data/processed/xgboost_results.csv')
    print("="*60)
    print("WEEK 2 XGBOOST RESULTS (BASELINE)")
    print("="*60)
    print(f"\nTraining Accuracy:  {week2_results['train_accuracy'].values[0]:.4f}")
    print(f"Testing Accuracy:   {week2_results['test_accuracy'].values[0]:.4f}")
    print(f"Precision (Macro):  {week2_results['precision_macro'].values[0]:.4f}")
    print(f"Recall (Macro):     {week2_results['recall_macro'].values[0]:.4f}")
    print(f"F1-Score (Macro):   {week2_results['f1_macro'].values[0]:.4f}")
    print("="*60)
    
    baseline_accuracy = week2_results['test_accuracy'].values[0]
    baseline_f1 = week2_results['f1_macro'].values[0]
except FileNotFoundError:
    print("Week 2 results not found. Using default baseline.")
    baseline_accuracy = 0.994
    baseline_f1 = 0.994

## Step 3: Define Optuna Objective Function

The objective function is the heart of Bayesian optimization. It:
1. Receives hyperparameters suggested by Optuna
2. Trains an XGBoost model with those parameters
3. Returns a score (we want to maximize accuracy)

### Hyperparameter Search Space:

| Parameter | Range | Type | Description |
|-----------|-------|------|-------------|
| n_estimators | 100-500 | int | Number of boosting rounds |
| max_depth | 3-12 | int | Maximum tree depth |
| learning_rate | 0.01-0.3 | float (log) | Step size shrinkage |
| subsample | 0.5-1.0 | float | Row sampling ratio |
| colsample_bytree | 0.5-1.0 | float | Column sampling ratio |
| min_child_weight | 1-10 | int | Minimum sum of instance weight in child |
| gamma | 0-5 | float | Minimum loss reduction for split |
| reg_alpha | 1e-8 to 10 | float (log) | L1 regularization |
| reg_lambda | 1e-8 to 10 | float (log) | L2 regularization |

### Why These Ranges?

- **n_estimators (100-500):** Week 2 used 200; we explore wider range
- **max_depth (3-12):** Week 2 used 6; deeper trees may capture more patterns
- **learning_rate (0.01-0.3):** Log scale because small changes matter more at low values
- **subsample/colsample (0.5-1.0):** Standard range for regularization
- **min_child_weight (1-10):** Controls overfitting; higher = more conservative
- **gamma (0-5):** Pruning parameter; higher = more pruning
- **reg_alpha/reg_lambda:** L1/L2 regularization; log scale for wide range

### Cross-Validation Strategy:

We use **5-fold Stratified Cross-Validation** to:
- Get robust performance estimates
- Maintain class distribution in each fold
- Reduce variance in optimization

In [ ]:
def objective(trial):
    """
    Optuna objective function for XGBoost hyperparameter optimization.
    
    This function:
    1. Suggests hyperparameters from defined ranges
    2. Creates an XGBoost model with those parameters
    3. Evaluates using 5-fold cross-validation
    4. Returns mean accuracy (to be maximized)
    """
    
    # Define hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10, log=True),
        'objective': 'multi:softmax',
        'num_class': 3,
        'eval_metric': 'mlogloss',
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0
    }
    
    # Create model
    model = XGBClassifier(**params)
    
    # 5-fold stratified cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train_encoded, cv=cv, scoring='accuracy', n_jobs=-1)
    
    return scores.mean()

print("Objective function defined successfully!")
print("\nSearch space summary:")
print("  - n_estimators: [100, 500]")
print("  - max_depth: [3, 12]")
print("  - learning_rate: [0.01, 0.3] (log scale)")
print("  - subsample: [0.5, 1.0]")
print("  - colsample_bytree: [0.5, 1.0]")
print("  - min_child_weight: [1, 10]")
print("  - gamma: [0, 5]")
print("  - reg_alpha: [1e-8, 10] (log scale)")
print("  - reg_lambda: [1e-8, 10] (log scale)")

## Step 4: Run Bayesian Optimization

Now we run the optimization process. Optuna will:
1. Start with random exploration
2. Build a surrogate model of the objective function
3. Use TPE (Tree-structured Parzen Estimator) to suggest promising hyperparameters
4. Iteratively improve until we reach the trial limit

### Configuration:
- **n_trials = 50:** Number of different hyperparameter combinations to try
- **sampler = TPESampler:** Tree-structured Parzen Estimator (default, most effective)
- **direction = 'maximize':** We want to maximize accuracy

### Why 50 Trials?

- Enough to explore the search space effectively
- Bayesian optimization typically converges within 30-50 trials
- Balances computation time with optimization quality
- Can be increased for potentially better results

### Expected Runtime:
Approximately 10-20 minutes depending on hardware (each trial involves 5-fold CV).

In [ ]:
# Create Optuna study
print("="*60)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*60)
print(f"\nNumber of trials: 50")
print(f"Optimization metric: Accuracy (maximize)")
print(f"Cross-validation: 5-fold Stratified")
print(f"\nThis may take 10-20 minutes...\n")

# Record start time
start_time = time.time()

# Create and run study
study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    study_name='xgboost_bayesian_optimization'
)

# Run optimization with progress callback
def callback(study, trial):
    if (trial.number + 1) % 10 == 0:
        print(f"  Trial {trial.number + 1}/50 completed. Best so far: {study.best_value:.4f}")

study.optimize(objective, n_trials=50, callbacks=[callback], show_progress_bar=False)

# Record end time
optimization_time = time.time() - start_time

print(f"\n" + "="*60)
print("OPTIMIZATION COMPLETE!")
print("="*60)
print(f"\nTotal optimization time: {optimization_time/60:.2f} minutes")
print(f"Best cross-validation accuracy: {study.best_value:.4f}")

## Step 5: Analyze Optimization Results

Let's examine the best hyperparameters found by Bayesian optimization and understand why they might be optimal.

### What to Look For:

1. **Best Parameters:** Compare with Week 2 manual selection
2. **Parameter Importance:** Which hyperparameters had the most impact?
3. **Convergence:** Did the optimization converge to a stable solution?
4. **Trial History:** How did performance improve over trials?

In [ ]:
# Display best parameters
print("="*60)
print("BEST HYPERPARAMETERS FOUND")
print("="*60)

best_params = study.best_params
print("\nOptimal hyperparameters:")
for param, value in best_params.items():
    if isinstance(value, float):
        print(f"  {param}: {value:.6f}")
    else:
        print(f"  {param}: {value}")

print("\n" + "="*60)
print("COMPARISON: Week 2 vs Bayesian Optimization")
print("="*60)

week2_params = {
    'n_estimators': 200,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'gamma': 0,
    'reg_alpha': 0,
    'reg_lambda': 1
}

print(f"\n{'Parameter':<20} {'Week 2':<15} {'Bayesian Opt':<15}")
print("-"*50)
for param in week2_params.keys():
    w2_val = week2_params[param]
    bo_val = best_params.get(param, 'N/A')
    if isinstance(bo_val, float):
        print(f"{param:<20} {w2_val:<15} {bo_val:<15.4f}")
    else:
        print(f"{param:<20} {w2_val:<15} {bo_val:<15}")

## Step 6: Train Final Model with Best Parameters

Now we train the final XGBoost model using the optimal hyperparameters found by Bayesian optimization.

### Why Retrain?

During optimization, we used cross-validation which trains on subsets of data. Now we:
1. Train on the **full training set** for maximum learning
2. Evaluate on the **held-out test set** for unbiased performance estimate
3. Save the final model for deployment

In [ ]:
# Train final model with best parameters
print("="*60)
print("TRAINING FINAL MODEL WITH OPTIMAL PARAMETERS")
print("="*60)

final_params = {
    **best_params,
    'objective': 'multi:softmax',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': 0
}

# Train model
best_model = XGBClassifier(**final_params)
best_model.fit(X_train, y_train_encoded)

# Make predictions
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate metrics
train_acc = accuracy_score(y_train_encoded, y_train_pred)
test_acc = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

print("\nFinal Model Performance:")
print(f"  Training Accuracy:  {train_acc:.4f}")
print(f"  Testing Accuracy:   {test_acc:.4f}")
print(f"  Precision (Macro):  {test_precision:.4f}")
print(f"  Recall (Macro):     {test_recall:.4f}")
print(f"  F1-Score (Macro):   {test_f1:.4f}")

print("\n" + "="*60)
print("Classification Report:")
print("="*60)
print(classification_report(y_test_encoded, y_test_pred, target_names=label_encoder.classes_))

## Step 7: Compare with Week 2 Results

This is the key comparison: Did Bayesian optimization improve upon our Week 2 GridSearchCV results?

### Comparison Metrics:

1. **Performance:** Test accuracy and F1-score
2. **Efficiency:** Number of trials/combinations evaluated
3. **Search Space:** Range of parameters explored

### Expected Outcomes:

- **Similar or slightly better performance:** Week 2 already achieved 99.4% accuracy
- **Much better efficiency:** Bayesian optimization explores intelligently vs exhaustively
- **Wider search space:** Found parameters we might not have tried manually

In [ ]:
print("="*70)
print("COMPREHENSIVE COMPARISON: Week 2 vs Bayesian Optimization")
print("="*70)

print("\n" + "-"*70)
print("PERFORMANCE COMPARISON")
print("-"*70)
print(f"{'Metric':<25} {'Week 2 (GridSearch)':<20} {'Bayesian Opt':<20}")
print("-"*70)
print(f"{'Training Accuracy':<25} {baseline_accuracy:.4f}{'':<14} {train_acc:.4f}")
print(f"{'Testing Accuracy':<25} {baseline_accuracy:.4f}{'':<14} {test_acc:.4f}")
print(f"{'F1-Score (Macro)':<25} {baseline_f1:.4f}{'':<14} {test_f1:.4f}")

# Calculate improvement
acc_improvement = (test_acc - baseline_accuracy) * 100
f1_improvement = (test_f1 - baseline_f1) * 100

print("\n" + "-"*70)
print("IMPROVEMENT ANALYSIS")
print("-"*70)
print(f"Accuracy change: {acc_improvement:+.2f}%")
print(f"F1-Score change: {f1_improvement:+.2f}%")

if acc_improvement > 0:
    print("\n Bayesian Optimization IMPROVED performance!")
elif acc_improvement == 0:
    print("\n Bayesian Optimization achieved EQUAL performance.")
else:
    print("\n Bayesian Optimization did not improve performance.")
    print("   This can happen when the baseline is already near-optimal.")

print("\n" + "-"*70)
print("EFFICIENCY COMPARISON")
print("-"*70)
print(f"Week 2 GridSearchCV: Exhaustive search (all combinations)")
print(f"Bayesian Optimization: 50 intelligent trials")
print(f"Optimization time: {optimization_time/60:.2f} minutes")
print("="*70)

## Step 8: Visualization - Optimization History

Visualizing the optimization process helps us understand:
1. How quickly the algorithm found good solutions
2. Whether it converged or was still improving
3. The distribution of trial performances

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Optimization History
ax1 = axes[0, 0]
trials_df = study.trials_dataframe()
ax1.plot(trials_df['number'], trials_df['value'], 'b-', alpha=0.5, label='Trial Accuracy')
ax1.scatter(trials_df['number'], trials_df['value'], c='blue', alpha=0.6, s=30)

# Best value line
best_values = [max(trials_df['value'][:i+1]) for i in range(len(trials_df))]
ax1.plot(trials_df['number'], best_values, 'r-', linewidth=2, label='Best So Far')
ax1.axhline(y=baseline_accuracy, color='green', linestyle='--', label=f'Week 2 Baseline ({baseline_accuracy:.4f})')

ax1.set_xlabel('Trial Number', fontsize=11)
ax1.set_ylabel('Cross-Validation Accuracy', fontsize=11)
ax1.set_title('Optimization History', fontsize=12, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# 2. Parameter Importance
ax2 = axes[0, 1]
param_importance = optuna.importance.get_param_importances(study)
params_sorted = sorted(param_importance.items(), key=lambda x: x[1], reverse=True)
param_names = [p[0] for p in params_sorted]
param_values = [p[1] for p in params_sorted]

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(param_names)))
bars = ax2.barh(param_names, param_values, color=colors)
ax2.set_xlabel('Importance', fontsize=11)
ax2.set_title('Hyperparameter Importance', fontsize=12, fontweight='bold')
ax2.invert_yaxis()

# 3. Trial Value Distribution
ax3 = axes[1, 0]
ax3.hist(trials_df['value'], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
ax3.axvline(x=study.best_value, color='red', linestyle='--', linewidth=2, label=f'Best: {study.best_value:.4f}')
ax3.axvline(x=baseline_accuracy, color='green', linestyle='--', linewidth=2, label=f'Week 2: {baseline_accuracy:.4f}')
ax3.set_xlabel('Cross-Validation Accuracy', fontsize=11)
ax3.set_ylabel('Frequency', fontsize=11)
ax3.set_title('Distribution of Trial Accuracies', fontsize=12, fontweight='bold')
ax3.legend()

# 4. Comparison Bar Chart
ax4 = axes[1, 1]
methods = ['Week 2\n(GridSearch)', 'Week 3\n(Bayesian Opt)']
accuracies = [baseline_accuracy, test_acc]
f1_scores = [baseline_f1, test_f1]

x = np.arange(len(methods))
width = 0.35

bars1 = ax4.bar(x - width/2, accuracies, width, label='Accuracy', color='steelblue')
bars2 = ax4.bar(x + width/2, f1_scores, width, label='F1-Score', color='coral')

ax4.set_ylabel('Score', fontsize=11)
ax4.set_title('Performance Comparison', fontsize=12, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(methods)
ax4.legend()
ax4.set_ylim(0.9, 1.0)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax4.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=9)
for bar in bars2:
    height = bar.get_height()
    ax4.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../../outputs/figures/bayesian_optimization_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved to: ../../outputs/figures/bayesian_optimization_analysis.png")

## Step 9: Save Results and Model

We save all important outputs for documentation and future use:

1. **Optimized model** - For deployment
2. **Best parameters** - For reproducibility
3. **Trial history** - For analysis
4. **Comparison results** - For reporting

In [ ]:
# Save best model
with open('../../models/xgboost_bayesian_optimized.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save best parameters
best_params_df = pd.DataFrame([best_params])
best_params_df.to_csv('../../outputs/bayesian_best_params.csv', index=False)

# Save trial history
trials_df = study.trials_dataframe()
trials_df.to_csv('../../outputs/bayesian_optimization_trials.csv', index=False)

# Save comparison results
comparison_df = pd.DataFrame({
    'Method': ['Week 2 (GridSearch)', 'Week 3 (Bayesian Optimization)'],
    'Test_Accuracy': [baseline_accuracy, test_acc],
    'F1_Score': [baseline_f1, test_f1],
    'n_estimators': [200, best_params['n_estimators']],
    'max_depth': [6, best_params['max_depth']],
    'learning_rate': [0.1, best_params['learning_rate']],
    'subsample': [0.8, best_params['subsample']],
    'colsample_bytree': [0.8, best_params['colsample_bytree']]
})
comparison_df.to_csv('../../outputs/bayesian_vs_gridsearch_comparison.csv', index=False)

# Save predictions
predictions_df = pd.DataFrame({
    'y_true': y_test_encoded,
    'y_pred': y_test_pred
})
predictions_df.to_csv('../../data/processed/xgboost_bayesian_predictions.csv', index=False)

print("="*60)
print("FILES SAVED SUCCESSFULLY")
print("="*60)
print("\nModel:")
print("  - ../../models/xgboost_bayesian_optimized.pkl")
print("\nResults:")
print("  - ../../outputs/bayesian_best_params.csv")
print("  - ../../outputs/bayesian_optimization_trials.csv")
print("  - ../../outputs/bayesian_vs_gridsearch_comparison.csv")
print("  - ../../data/processed/xgboost_bayesian_predictions.csv")
print("\nVisualization:")
print("  - ../../outputs/figures/bayesian_optimization_analysis.png")

## Conclusion

### Summary

In this task, we implemented **Bayesian Optimization using Optuna** to find optimal XGBoost hyperparameters and compared the results with Week 2's GridSearchCV approach.

### Key Findings

1. **Performance:** Bayesian optimization achieved comparable or improved results compared to Week 2
2. **Efficiency:** Explored a much larger search space with only 50 intelligent trials
3. **Parameter Insights:** Identified which hyperparameters have the most impact on model performance

### Advantages of Bayesian Optimization

| Advantage | Description |
|-----------|-------------|
| **Intelligent Search** | Learns from previous trials to focus on promising regions |
| **Efficiency** | Finds good solutions with fewer evaluations |
| **Continuous Parameters** | Handles continuous ranges naturally (no discretization needed) |
| **Scalability** | Works well even with many hyperparameters |
| **Uncertainty Quantification** | Balances exploration and exploitation |

### When to Use Bayesian Optimization

- Large hyperparameter search spaces
- Expensive model training (each evaluation is costly)
- Continuous hyperparameters
- When GridSearchCV is too slow

### Limitations

- May not significantly improve already near-optimal models (like our 99.4% baseline)
- Requires more setup than simple GridSearchCV
- Results can vary between runs (though we use seeds for reproducibility)

### Next Steps

- Apply similar optimization to Random Forest (Task 3.8)
- Use the optimized model for SHAP analysis (Task 3.13)
- Include these results in the final report (Task 3.18)